## Indian Premier League (IPL) Regression Analysis

In [1]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [2]:
# As usual, we begin by loading the packages we will need

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

### 1. Load the data

In [3]:
# Now we load the data

IPL=pd.read_excel("Assignment Data/Week 5/IPL (assignment) data.xlsx")

In [4]:
IPL.head()

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000


### 2. Create the sum of salaries in each season

In [6]:
Sumsal = IPL.groupby(['year'])['salaries'].sum().reset_index().rename(columns = {'salaries':'allsal'})
Sumsal

,year,allsal
0,2008,34105000
1,2009,33445000
2,2010,33075000
3,2011,62210000
4,2012,59706250
5,2013,81535000
6,2014,73973332
7,2015,65242665
8,2016,63483666
9,2017,63014833


In [7]:
IPL = pd.merge(IPL, Sumsal, on = ['year'], how = 'left')
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833


### 3. Create a variable for team salary divided by total salaries for that season (relsal).

In [8]:
IPL['relsal'] = IPL['salaries']/IPL['allsal']
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal,relsal
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332,0.132525
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665,0.124223
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666,0.137794
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833,0.130136


### 4. Create a value for win percentage. Define win percentage as wins divided games with a result (= games played minus games with no result). 

In [9]:
IPL['wpc'] = IPL['won']/(IPL['played'] - IPL['noresult'])
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal,relsal,wpc
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796,0.571429
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272,0.615385
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846,0.500000
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752,0.642857
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332,0.132525,0.428571
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665,0.124223,0.500000
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666,0.137794,0.571429
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833,0.130136,0.615385


### 5. Create the lagged value of win percentage for each team

In [10]:
IPL['wpc_lag'] = IPL.groupby('team')['wpc'].shift(1)
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal,relsal,wpc,wpc_lag
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796,0.571429,NaN
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272,0.615385,0.571429
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846,0.500000,0.615385
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752,0.642857,0.500000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314,0.533333,0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332,0.132525,0.428571,0.625000
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665,0.124223,0.500000,0.428571
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666,0.137794,0.571429,0.500000
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833,0.130136,0.615385,0.571429


### 6. Regress win percentage on:
a) Relsal <br> 
b) Relsal + lagged win percentage <br> 
c) Relsal + lagged win percentage  + team fixed effects

In [11]:
wpcsal1 = smf.ols(formula = 'wpc ~ relsal', data = IPL).fit() 
wpcsal1.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    wpc   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.1622
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.688
Time:                        08:47:02   Log-Likelihood:                 45.259
No. Observations:                  92   AIC:                            -86.52
Df Residuals:                      90   BIC:                            -81.47
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4752      0.063      7.563      0.000       0.350       0.600
relsal         0.2050      0.509      0.403      0.688      -0.806       1.216
==============================================================================
Omnibus:                        3.778   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.151   Jarque-Bera (JB):                3.780
Skew:                          -0.474   Prob(JB):                        0.151
Kurtosis:                       2.703   Cond. No.                         33.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
wpcsal2 = smf.ols(formula = 'wpc ~ relsal + wpc_lag', data = IPL).fit() 
wpcsal2.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    wpc   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.591
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.210
Time:                        08:47:40   Log-Likelihood:                 43.314
No. Observations:                  79   AIC:                            -80.63
Df Residuals:                      76   BIC:                            -73.52
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3489      0.087      4.003      0.000       0.175       0.522
relsal         0.6785      0.523      1.297      0.198      -0.363       1.720
wpc_lag        0.1406      0.108      1.300      0.198      -0.075       0.356
==============================================================================
Omnibus:                        3.816   Durbin-Watson:                   1.827
Prob(Omnibus):                  0.148   Jarque-Bera (JB):                3.581
Skew:                          -0.520   Prob(JB):                        0.167
Kurtosis:                       2.926   Cond. No.                         37.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
wpcsal3 = smf.ols(formula = 'wpc ~ wpc_lag + relsal + C(team)', data = IPL).fit() 
wpcsal3.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    wpc   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     2.117
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0245
Time:                        08:48:14   Log-Likelihood:                 55.639
No. Observations:                  79   AIC:                            -83.28
Df Residuals:                      65   BIC:                            -50.10
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.6665      0.138      4.826      0.000       0.391       0.942
C(team)[T.Deccan Chargers]                -0.1857      0.085     -2.187      0.032      -0.355      -0.016
C(team)[T.Delhi Daredevils]               -0.2059      0.069     -2.972      0.004      -0.344      -0.068
C(team)[T.Gujarat Lions]                  -0.3294      0.143     -2.310      0.024      -0.614      -0.045
C(team)[T.Kings XI Punjab]                -0.1747      0.069     -2.547      0.013      -0.312      -0.038
C(team)[T.Kochi Tuskers Kerala]             6e-17   2.77e-16      0.216      0.829   -4.94e-16    6.14e-16
C(team)[T.Kolkata Knight Riders]          -0.0972      0.065     -1.495      0.140      -0.227       0.033
C(team)[T.Mumbai Indians]                 -0.0405      0.065     -0.622      0.536      -0.170       0.090
C(team)[T.Pune Warriors India]            -0.3893      0.118     -3.301      0.002      -0.625      -0.154
C(team)[T.Rajasthan Royals]               -0.1259      0.086     -1.471      0.146      -0.297       0.045
C(team)[T.Rising Pune Supergiants]         0.0118      0.144      0.082      0.935      -0.275       0.299
C(team)[T.Royal Challengers Bangalore]    -0.1175      0.064     -1.838      0.071      -0.245       0.010
C(team)[T.Sunrisers Hyderabad]            -0.0669      0.076     -0.877      0.384      -0.219       0.086
wpc_lag                                   -0.0636      0.115     -0.554      0.582      -0.293       0.166
relsal                                    -0.1014      0.690     -0.147      0.884      -1.478       1.276
==============================================================================
Omnibus:                        2.352   Durbin-Watson:                   2.188
Prob(Omnibus):                  0.308   Jarque-Bera (JB):                1.775
Skew:                          -0.153   Prob(JB):                        0.412
Kurtosis:                       3.668   Cond. No.                     8.54e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.49e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""